In [1]:
!verdi daemon restart
!reentry scan

Restarting the daemon... OK


In [2]:
import aiida
import os.path as op
from aiida import orm
from aiida.orm import load_code,Dict,StructureData,KpointsData,Int,Str,List,SinglefileData,Float
from aiida.engine import submit
from aiida_pseudo.data.pseudo.psf import PsfData

aiida.load_profile()

# Structure

In [3]:
import sisl
Si64_fdf = sisl.get_sile('/media/aakhtar/AiiDA/siesta-master/Obj_lua/Tests/si64/si64.fdf')
Si64_geom = Si64_fdf.read_geometry()
Si64_geom_ase = Si64_geom.toASE()
Si64_geom_ase.write('host.XSF')
host = StructureData(ase=Si64_geom_ase)
iv=0 
defect = host.clone()
defect.clear_sites()
sites =host.sites
new_sites = sites
vacancy_site = new_sites.pop(iv)
[defect.append_site(s) for s in new_sites]
#len(host.sites)
#len(defect.sites)
ase_defect = defect.get_ase()
ase_defect.write('defect.XSF')

In [4]:
from aiida_defects.formation_energy_siesta.formation_energy_siesta_manual_wk_node import FormationEnergyWorkchainSIESTAManual

In [5]:
# The pseudopotentials
host_pseudos_dict = {}
#raw_pseudos = [("Si.psf", ['Si']), ("H.psf", ['H_int'])]
raw_pseudos = [("Si.psf", ['Si'])]
for fname, kinds in raw_pseudos:
    #absname = op.realpath(op.join(op.dirname(__file__), "fixtures/sample_psf", fname))
    absname = op.join("/media/aakhtar/AiiDA/aiida-siesta-barrier/aiida_siesta_barrier/examples/fixtures/sample_psf", fname)
    pseudo = PsfData.get_or_create(absname)
    if not pseudo.is_stored:
        print("\nCreated the pseudo for {}".format(kinds))
    else:
        print("\nUsing the pseudo for {} from DB: {}".format(kinds, pseudo.pk))
    for j in kinds:
        host_pseudos_dict[j]=pseudo


Using the pseudo for ['Si'] from DB: 53


In [6]:
# The pseudopotentials
defect_pseudos_dict = {}
#raw_pseudos = [("Si.psf", ['Si']), ("H.psf", ['H_int'])]
raw_pseudos = [("Si.psf", ['Si','Si_ghost'])]
for fname, kinds in raw_pseudos:
    #absname = op.realpath(op.join(op.dirname(__file__), "fixtures/sample_psf", fname))
    absname = op.join("/media/aakhtar/AiiDA/aiida-siesta-barrier/aiida_siesta_barrier/examples/fixtures/sample_psf", fname)
    pseudo = PsfData.get_or_create(absname)
    if not pseudo.is_stored:
        print("\nCreated the pseudo for {}".format(kinds))
    else:
        print("\nUsing the pseudo for {} from DB: {}".format(kinds, pseudo.pk))
    for j in kinds:
        defect_pseudos_dict[j]=pseudo


Using the pseudo for ['Si', 'Si_ghost'] from DB: 53


In [7]:
# Parameters: very coarse for speed of test
# Note the all the Si atoms are fixed...
parameters = dict={
   "mesh-cutoff": "50 Ry",
   "dm-tolerance": "0.001",
   "DM-NumberPulay ":  "3",
   "DM-History-Depth":  "0",
   "SCF-Mixer-weight":  "0.02",
   "SCF-Mix":   "density",
   "SCF-Mixer-kick":  "35",
   "MD-VariableCell":  "F",
   "MD-MaxCGDispl":  "0.3 Bohr",
   "MD-MaxForceTol":  " 0.04000 eV/Ang"
    }
defect_constraints = dict={
    "%block geometryconstraints":
    """
    species-i 2 
    %endblock geometryconstraints"""
    }
relaxation = dict={
    'md-steps': 100
    }
parameters.update(relaxation)
#
# Use this for constraints
#
#parameters.update(constraints)
#
host_parameters = Dict(dict=parameters)
defect_q0_parameters = Dict(dict=parameters)
defect_q_parameters = Dict(dict=parameters)

In [8]:
host_parameters.get_dict()

{'mesh-cutoff': '50 Ry',
 'dm-tolerance': '0.001',
 'DM-NumberPulay ': '3',
 'DM-History-Depth': '0',
 'SCF-Mixer-weight': '0.02',
 'SCF-Mix': 'density',
 'SCF-Mixer-kick': '35',
 'MD-VariableCell': 'F',
 'MD-MaxCGDispl': '0.3 Bohr',
 'MD-MaxForceTol': ' 0.04000 eV/Ang',
 'md-steps': 100}

In [9]:
defect_q0_parameters.update_dict(defect_constraints)
defect_q0_parameters.get_dict()

{'mesh-cutoff': '50 Ry',
 'dm-tolerance': '0.001',
 'DM-NumberPulay ': '3',
 'DM-History-Depth': '0',
 'SCF-Mixer-weight': '0.02',
 'SCF-Mix': 'density',
 'SCF-Mixer-kick': '35',
 'MD-VariableCell': 'F',
 'MD-MaxCGDispl': '0.3 Bohr',
 'MD-MaxForceTol': ' 0.04000 eV/Ang',
 'md-steps': 100,
 '%block geometryconstraints': '\n    species-i 2 \n    %endblock geometryconstraints'}

In [10]:
defect_q_parameters.update_dict(defect_constraints)
defect_q_parameters.get_dict()

{'mesh-cutoff': '50 Ry',
 'dm-tolerance': '0.001',
 'DM-NumberPulay ': '3',
 'DM-History-Depth': '0',
 'SCF-Mixer-weight': '0.02',
 'SCF-Mix': 'density',
 'SCF-Mixer-kick': '35',
 'MD-VariableCell': 'F',
 'MD-MaxCGDispl': '0.3 Bohr',
 'MD-MaxForceTol': ' 0.04000 eV/Ang',
 'md-steps': 100,
 '%block geometryconstraints': '\n    species-i 2 \n    %endblock geometryconstraints'}

In [11]:
net_charge = dict={'NetCharge':+2.0}

In [12]:
net_charge

{'NetCharge': 2.0}

In [13]:
inputs_test = Dict(dict=defect_q_parameters.get_dict())

In [14]:
inputs_test

<Dict: uuid: 9a4d4151-b11a-47b5-9b55-12c004ca6a5e (unstored)>

In [15]:
inputs_test.update_dict(net_charge)

In [16]:
basis = Dict(
    dict={
        'pao-energy-shift':
        '300 meV',
        '%block pao-basis-sizes':
        """
        Si SZ
        %endblock pao-basis-sizes""",})

basis_defect = Dict(
    dict={
        'pao-energy-shift':
        '300 meV',
        '%block pao-basis-sizes':
        """
        Si SZ
        Si_ghost SZ
        %endblock pao-basis-sizes""",})

In [17]:
#Resources
options = Dict(dict={
    "max_wallclock_seconds": 73600,
    'withmpi': True,
    "resources": {
        "num_machines": 1,
        "num_mpiprocs_per_machine": 1,
    }
})


In [18]:
options.get_dict()

{'max_wallclock_seconds': 73600,
 'withmpi': True,
 'resources': {'num_machines': 1, 'num_mpiprocs_per_machine': 1}}

In [19]:
floating = {'floating_sites': [{"symbols": 'Si',
                                "name": 'Si_ghost',
                               "position": vacancy_site.position}]}

In [20]:
basis_defect.update_dict(floating)

In [21]:
basis_defect.get_dict()

{'pao-energy-shift': '300 meV',
 '%block pao-basis-sizes': '\n        Si SZ\n        Si_ghost SZ\n        %endblock pao-basis-sizes',
 'floating_sites': [{'symbols': 'Si',
   'name': 'Si_ghost',
   'position': (0.0, 0.0, 0.0)}]}

In [22]:
codename = 'siesta-master@arsalan-pwk'
code = load_code(codename)

In [23]:
#The kpoints
kpoints = KpointsData()
kpoints.set_kpoints_mesh([1,1,1])

In [29]:
inputs = {'host_structure':host,
          'defect_structure':defect,
    'host': {'code':code,
             'parameters':host_parameters,
             'basis':basis,
             'kpoints':kpoints,
             'pseudos':host_pseudos_dict,
             'options':options},
    'defect_q0': {'code':code,
                  'parameters':defect_q0_parameters,
                  'basis':basis_defect,
                  'kpoints':kpoints,
                  'pseudos':defect_pseudos_dict,
                  'options':options},
    'defect_q': {'code':code,
                 'parameters':defect_q_parameters,
                 'basis':basis_defect,
                 'kpoints':kpoints,
                 'pseudos':defect_pseudos_dict,
                 'options':options},
    'defect_charge':Float(-1),
    'defect_site':List(list=[0.0,0.0,0.0]),
    'chemical_potential':Float(-100.629519),
    'epsilon':Float(9.0),
    'wk_node':Int(387),
    'add_or_remove':Str('remove'),
}



In [30]:
inputs

{'host_structure': <StructureData: uuid: 38635215-8f3a-4950-84e5-3ed012c2161b (pk: 586)>,
 'defect_structure': <StructureData: uuid: e1abc0f1-17da-4c7d-b4ed-bd3fe686d6c7 (pk: 587)>,
 'host': {'code': <Code: Remote code 'siesta-master' on arsalan-pwk, pk: 1, uuid: 1fac24eb-3d04-4037-b543-5754209df3e0>,
  'parameters': <Dict: uuid: 1f0ff761-6f10-4928-937d-d6f093ebde74 (pk: 588)>,
  'basis': <Dict: uuid: 208d7f42-e1ed-4d5b-8e08-1bc79b07189a (pk: 589)>,
  'kpoints': <KpointsData: uuid: de968881-fce1-4a5e-897d-805d946f4009 (pk: 590)>,
  'pseudos': {'Si': <PsfData: uuid: 8927ec4e-7fb2-487d-8aab-3a4fefcb726a (pk: 53)>},
  'options': <Dict: uuid: bebfadc6-c8f5-4cdf-9a54-b91b91795a94 (pk: 591)>},
 'defect_q0': {'code': <Code: Remote code 'siesta-master' on arsalan-pwk, pk: 1, uuid: 1fac24eb-3d04-4037-b543-5754209df3e0>,
  'parameters': <Dict: uuid: 4c477cb0-91a4-4160-936e-44cde9cb1ef8 (pk: 594)>,
  'basis': <Dict: uuid: 8c6b1732-eeb7-4185-b42c-3735feda90b3 (pk: 595)>,
  'kpoints': <KpointsData:

In [31]:
process = submit(FormationEnergyWorkchainSIESTAManual, **inputs)
print("Submitted VacancyExchangeBarrier workchain; ID={}".format(process.pk))
print(
    "For information about this workchain type: verdi process show {}".format(
        process.pk))
print("For a list of running processes type: verdi process list")


Submitted VacancyExchangeBarrier workchain; ID=636
For information about this workchain type: verdi process show 636
For a list of running processes type: verdi process list


In [27]:
node =orm.load_node(387)